#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2020


# Implicit Library in Python

### Spotlight Author - Anna Shekhawat (UIN: 930001909)

The Implicit library, written by Ben Frederickson, has been designed to contain fast implementations of many popular recommendation algorithms. And these are mainly for implicit feedback datasets. Some of the algorithms are alternating least squares, Bayesian personalized ranking, logistic matrix factorization, and nearest neighbor models.

To make the implementations fast, one of the things the developers have used is multi-threading to make use of all the available cores. They use Cython and OpenMP to run the models in parallel. And a comparison of these implementations against the basic Python single thread implementation definitely shows that the Implicit library implementations can be a 1000 times faster.

**In this spotlight:**

*Main focus* - Implicit library

*Dataset used* - Online Retail dataset from the UCI Machine Learning Repository

*Algorithm for recommendation* - Alternating Least Squares (ALS)

I will go through an example of building a recommender system for a dataset from the UCI Machine Learning repository. The dataset can be found at this link - https://archive.ics.uci.edu/ml/datasets/Online+Retail
It consists of some online retail data, like customer id, item id, and the units of those items that this customer bought, date on which they were bought and other similar details. But the most important to us is going to be the customer and item id and one such pair would indicate that this customer bought this item, and we'll use it as implicit feedback to make recommendations to that user about other items.

**How Alternating Least Squares works:**

The recommendation algorithm I am using is the Alternating Least Squares (ALS) algorithm. This algorithm is a way to do matrix factorization. Let's say we have an implicit data matrix N x M with N users and M items. We essentially want to factorize this matrix into two matrices:

one matrix U, of size N x K, which has K latent feature vectors for each user,

and the other matrix, V, of size K x M, which has K latent feature vectors for each item.

Upon multiplication of these two, we can generate the prediction matrix, P, of size N x M which will be a dense matrix, containing a prediction value for each pair of user and item.

Since the original matrix is large and sparse, we don't want to invert it which is needed in the case of SVD. Instead, we can run an approximation algorithm to reach close to the result we would achieve with SVD.

In the algorithm, we first randomly initialize U and solve for V. Then from the V matrix we got, we solve for U again. We can keep iterating this process until a convergence level is reached. At a time, the algorithm solves for one feature vector. And this makes it a perfect candidate for running in parallel, which is what Ben Frederickson, the author of the library, Implicit, has built upon. this results in the training process running very fast.

All the imports used in this spotlight:

In [1]:
import implicit
import random
import pandas as pd
from pandas.api.types import CategoricalDtype
from scipy.sparse.linalg import spsolve
import scipy.sparse as sparse
import numpy as np
from sklearn.preprocessing import MinMaxScaler

## Data Preprocessing

Since the dataset is available in CSV format, it will be easy to fetch the data using Pandas and store it in a Pandas dataframe.

In [2]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
online_retail_data = pd.read_excel(url)

In [3]:
online_retail_data[:15]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


The above is the first fifteen rows of the data. We can see that the data includes invoice number, stock code, description of the item, quantity of the item purchased, the invoice date, per unit price of the item, customer id, and the country where the purchase was made. Clearly, rows 0 to 8 have the same invoice number, date and customer id. So those nine rows contain items that were bought together in a single purchase by the customer 17850. And after that, the items purchased together at some other time by some other user are listed. So far in these fifteen rows, the data looks dense without any missing values. But just to be sure, I'll use the info() method to check the overall shape of the data.

In [4]:
online_retail_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


We observe that some rows are missing customer ids. That is not going to work with matrix factorization. So we can get rid of all those rows.

In [5]:
online_retail_data = online_retail_data.loc[
    pd.isnull(online_retail_data.CustomerID) == False]
online_retail_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      406829 non-null object
StockCode      406829 non-null object
Description    406829 non-null object
Quantity       406829 non-null int64
InvoiceDate    406829 non-null datetime64[ns]
UnitPrice      406829 non-null float64
CustomerID     406829 non-null float64
Country        406829 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


Now the data looks alright to go ahead. No more values are missing.

Now I will create a table of unique stock codes and description pairs, so that later for analysis purposes, we can look at what item a stock code actually represents.

In [6]:
item_lookup = online_retail_data[['StockCode', 'Description']].drop_duplicates()
item_lookup['StockCode'] = item_lookup.StockCode.astype(str)
item_lookup.head()

,StockCode,Description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


Now following are some more data preprocessing steps and generating the sparse matrix. I want to use the sparse formay of matrix to avoid any unnecessary wastage of storage.

In [7]:
online_retail_data['CustomerID'] = online_retail_data.CustomerID.astype(int)
online_retail_data = online_retail_data[['StockCode', 'Quantity', 'CustomerID']]
grouped_and_cleaned_data = online_retail_data.groupby(['CustomerID', 'StockCode']).sum().reset_index()
grouped_and_cleaned_data.Quantity.loc[grouped_and_cleaned_data.Quantity == 0] = 1
grouped_and_purchased = grouped_and_cleaned_data.query('Quantity > 0')

grouped_and_purchased.head()

quantity = list(grouped_and_purchased.Quantity)
customers = list(np.sort(grouped_and_purchased.CustomerID.unique()))
products = list(grouped_and_purchased.StockCode.unique())

customers_rows = grouped_and_purchased.CustomerID.astype(CategoricalDtype(categories = customers)).cat.codes 
items_cols = grouped_and_purchased.StockCode.astype(CategoricalDtype(categories = products)).cat.codes 
all_purchases_sparse_matrix = sparse.csr_matrix((quantity, (customers_rows, items_cols)), shape=(len(customers), len(products)))

all_purchases_sparse_matrix

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


<4338x3664 sparse matrix of type '<class 'numpy.int64'>'
	with 266723 stored elements in Compressed Sparse Row format>

We observe that the actual matrix is of size 4338 users x 3664 items. But the sparse matrix only contains 266723 elements. We can use this information to compute the sparsity of the matrix.

In [8]:
matrix_size = all_purchases_sparse_matrix.shape[0] * all_purchases_sparse_matrix.shape[1]
number_of_purchases = len(all_purchases_sparse_matrix.nonzero()[0])
sparsity = 100*(1 - (number_of_purchases / matrix_size))
print('sparsity =', sparsity, '%')

sparsity = 98.32190920694744 %


So the matrix is 98.32% sparse. With this, we can easily do matrix factorization. It is above 99.5% that that becomes difficult. So we should be good to go.

The next step is to generate train and test sets.
The entire matrix size needs to be used for training. What we can do is mask some of the original ratings/values where a rating is present. By masking, I mean we can assign them a value of 0. For the test set, we can use all the original ratings we have. The masking_rate variable below indicates how many original ratings get masked per 1 rating.

In [9]:
def generate_train_test_sets(ratings, masking_rate = 0.25):
    # the entire dataset is the test_set:
    test_set = ratings.copy()
    # making the test set binary:
    test_set[test_set != 0] = 1
    training_set = ratings.copy()
    nonzero_indices = training_set.nonzero()
    nonzero_pairs = list(zip(nonzero_indices[0], nonzero_indices[1]))
    random.seed(0)
    samples_count = int(np.ceil(masking_rate * len(nonzero_pairs)))
    
    # sampling random customer-item pairs
    random_samples = random.sample(nonzero_pairs, samples_count)
    customer_indices = [index[0] for index in random_samples]
    item_indices = [index[1] for index in random_samples]
    training_set[customer_indices, item_indices] = 0
    training_set.eliminate_zeros()
    return training_set, test_set, list(set(customer_indices))

Call the above function to generate train and test sets with a masking rate of 25%.

In [10]:
training_set, test_set, customer_indices = generate_train_test_sets(all_purchases_sparse_matrix, masking_rate = 0.25)

## Training

The following code will train the model. I have set K = 20 in this case. It means there will be 20 latent factors for each user and item. Let's call it the 20 latent factors model.

In [37]:
user_vecs_20, item_vecs_20 = implicit.alternating_least_squares((training_set * 15).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

Now let's train another model. This will have 100 latent factors which is five times the latent factors in the previous model. This should be able to capture the features better.

In [38]:
user_vecs_100, item_vecs_100 = implicit.alternating_least_squares((training_set * 15).astype('double'), 
                                                          factors=100, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

## Testing

As can be seen from the abvove two outputs, nearly 3-4 iterations finished in a second. A normal Python implementation that doesn't run in parallel takes around 45 seconds for a single iteration to finish. So the value added by the Implicit library implemntation is immense.

Now let's write a recommendation helper function to help us pick top items to recommend to a user.

In [28]:
def recommend_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    
    cust_ind = np.where(customer_list == customer_id)[0][0]
    pref_vec = mf_train[cust_ind,:].toarray()
    pref_vec = pref_vec.reshape(-1) + 1 
    pref_vec[pref_vec > 1] = 0
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T)
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] 
    rec_list = [] 
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.Description.loc[item_lookup.StockCode == code].iloc[0]]) 
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'StockCode': codes, 'Description': descriptions})
    return final_frame[['StockCode', 'Description']]

Now that we have our trained model and the testing and recommendation code ready, we can do some test runs and try analyse the results logically.

For this, we can select a customer and see what items that customer has purchased in the past, then generate recommended items for this customer using our trained model, and see if there's any connection between the two. To do the same, I am first generating an array of customers only, another array of items only, and I am writing a function that returns a list of items that have been purchased by a given customer.

In [14]:
customers_arr = np.array(customers)
products_arr = np.array(products)

In [15]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    cust_ind = np.where(customers_list == customer_id)[0][0]
    purchased_ind = mf_train[cust_ind,:].nonzero()[1]
    prod_codes = products_list[purchased_ind]
    return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]

In [16]:
customers_arr[:10]

array([12346, 12347, 12348, 12349, 12350, 12352, 12353, 12354, 12355,
       12356])

Above is a list of first ten customers in the customer array. Let's check the items bought by the customer 12356, for example.

In [17]:
get_items_purchased(12356, training_set, customers_arr, products_arr, item_lookup)

,StockCode,Description
45,POST,POSTAGE
96,21212,PACK OF 72 RETROSPOT CAKE CASES
99,84991,60 TEATIME FAIRY CAKE CASES
134,22646,CERAMIC STRAWBERRY CAKE MONEY BANK
209,22195,LARGE HEART MEASURING SPOONS
224,21080,SET/20 RED RETROSPOT PAPER NAPKINS
226,21086,SET/6 RED SPOTTY PAPER CUPS
347,21122,SET/10 PINK POLKADOT PARTY CANDLES
352,22435,SET OF 9 HEART SHAPED BALLOONS
394,21125,SET 6 FOOTBALL CELEBRATION CANDLES


Going through the list above, it is easy to notice that this customer bought a lot of party material like decorations, cakes, candles, snacks, cutlery.

Now let's run the learned 20 latent factors recommendation model for this customer and see what items are the top 20 choices to be recommended to this customer.

In [31]:
recommend_items(12356, training_set, user_vecs_20, item_vecs_20, customers_arr, products_arr, item_lookup,
                       num_items = 20)

,StockCode,Description
0,84568,GIRLS ALPHABET IRON ON PATCHES
1,37446,MINI CAKE STAND WITH HANGING CAKES
2,37450,CERAMIC CAKE BOWL + HANGING CAKES
3,21986,PACK OF 12 PINK POLKADOT TISSUES
4,21700,BIG DOUGHNUT FRIDGE MAGNETS
5,85215,ASSORTED CHEESE FRIDGE MAGNETS
6,21211,SET OF 72 SKULL PAPER DOILIES
7,21126,SET OF 6 GIRLS CELEBRATION CANDLES
8,21975,PACK OF 60 DINOSAUR CAKE CASES
9,22057,CERAMIC PLATE STRAWBERRY DESIGN


The above list of recommendations seems fitting to what the customer had already bought. The recommendations include some party materials like cakes, storage containers, party bags, plates for cake, decorations, candles. So the algorithm clearly does a good job for this customer.

Now, let's try the 100 latent factors model.

In [33]:
recommend_items(12356, training_set, user_vecs_100, item_vecs_100, customers_arr, products_arr, item_lookup,
                       num_items = 20)

,StockCode,Description
0,21355,TOAST ITS - I LOVE YOU
1,22851,SET 20 NAPKINS FAIRY CAKES DESIGN
2,72741,GRAND CHOCOLATECANDLE
3,84536B,FAIRY CAKES NOTEBOOK A7 SIZE
4,47599B,BLUE PARTY BAGS
5,21354,TOAST ITS - BEST MUM
6,84970L,SINGLE HEART ZINC T-LIGHT HOLDER
7,37450,CERAMIC CAKE BOWL + HANGING CAKES
8,22335,HEART DECORATION PAINTED ZINC
9,22645,CERAMIC HEART FAIRY CAKE MONEY BANK


It is easy to notice that this recommendations list is even closer to customer's puchase history than the 20 latent factors model one. The 100 latent factors model definitely does a better job capturing the user's choices as compared to the 20 latent factors model.

Let's try another customer, say customer 12350. First let's get the items actually purchased by this customer.

In [20]:
get_items_purchased(12350, training_set, customers_arr, products_arr, item_lookup)

,StockCode,Description
45,POST,POSTAGE
144,21832,CHOCOLATE CALCULATOR
240,22557,PLASTERS IN TIN VINTAGE PAISLEY
439,21915,RED HARMONICA IN BOX
480,22620,4 TRADITIONAL SPINNING TOPS
494,21866,UNION JACK FLAG LUGGAGE TAG
2091,79066K,RETRO MOD TRAY
2301,21864,UNION JACK FLAG PASSPORT COVER
3537,79191C,RETRO PLASTIC ELEPHANT TRAY
5137,20615,BLUE POLKADOT PASSPORT COVER


The customer has bought some home decor items, some bags/covers, also teabags. Another noticeable quality of items is their distinct design/pattern, like the paisley, polkadot, vintage design. Now, let's run the 20 latent factor prediction model to generate recommendations for this customer.

In [21]:
recommend_items(12350, training_set, user_vecs_20, item_vecs_20, customers_arr, products_arr, item_lookup,
                       num_items = 20)

,StockCode,Description
0,22197,SMALL POPCORN HOLDER
1,22616,PACK OF 12 LONDON TISSUES
2,84879,ASSORTED COLOUR BIRD ORNAMENT
3,21986,PACK OF 12 PINK POLKADOT TISSUES
4,85099B,JUMBO BAG RED RETROSPOT
5,22614,PACK OF 12 SPACEBOY TISSUES
6,15036,ASSORTED COLOURS SILK FAN
7,20712,JUMBO BAG WOODLAND ANIMALS
8,23204,CHARLOTTE BAG APPLES DESIGN
9,21824,PAINTED METAL STAR WITH HOLLY BELLS


As can be seen from the list above, the recommendations are very close to what the customer is known to have bought historically The recommendations include the vintage, polkadot, and paisley designs, it includes some bags, some tea related items, some home decor items. So once again, the algorithm generates very good recommendations.

Now let's see the recommendations of the 100 latent factors model.

In [34]:
recommend_items(12350, training_set, user_vecs_100, item_vecs_100, customers_arr, products_arr, item_lookup,
                       num_items = 20)

,StockCode,Description
0,23199,JUMBO BAG APPLES
1,22386,JUMBO BAG PINK POLKADOT
2,85099F,JUMBO BAG STRAWBERRY
3,22383,LUNCH BAG SUKI DESIGN
4,20713,JUMBO BAG OWLS
5,85099B,JUMBO BAG RED RETROSPOT
6,23206,LUNCH BAG APPLE DESIGN
7,21929,JUMBO BAG PINK VINTAGE PAISLEY
8,20712,JUMBO BAG WOODLAND ANIMALS
9,21930,JUMBO STORAGE BAG SKULLS


Again, the list is very relevant to the customer's purchase history.

Let's try one final example. First, let's get the list of items actually purchased by customer 12355.

In [23]:
get_items_purchased(12355, training_set, customers_arr, products_arr, item_lookup)

,StockCode,Description
500,72802C,VANILLA SCENT CANDLE JEWELLED BOX
880,22423,REGENCY CAKESTAND 3 TIER
1077,22699,ROSES REGENCY TEACUP AND SAUCER
1086,22697,GREEN REGENCY TEACUP AND SAUCER
5014,22649,STRAWBERRY FAIRY CAKE TEAPOT
5108,22890,NOVELTY BISCUITS CAKE STAND 3 TIER
7116,72802B,OCEAN SCENT CANDLE IN JEWELLED BOX
19923,22698,PINK REGENCY TEACUP AND SAUCER
26685,85040A,S/4 PINK FLOWER CANDLES IN BOWL
131403,23077,DOUGHNUT LIP GLOSS


The customer has purchased jewelled boxes, teapot, saucer, candles, lip gloss.

Let's run the 20 latent factor model.

In [39]:
recommend_items(12355, training_set, user_vecs_20, item_vecs_20, customers_arr, products_arr, item_lookup,
                       num_items = 20)

,StockCode,Description
0,72741,GRAND CHOCOLATECANDLE
1,22693,GROW A FLYTRAP OR SUNFLOWER IN TIN
2,22646,CERAMIC STRAWBERRY CAKE MONEY BANK
3,22645,CERAMIC HEART FAIRY CAKE MONEY BANK
4,21232,STRAWBERRY CERAMIC TRINKET BOX
5,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR
6,23078,ICE CREAM PEN LIP GLOSS
7,22644,CERAMIC CHERRY CAKE MONEY BANK
8,21231,SWEETHEART CERAMIC TRINKET BOX
9,20828,GLITTER BUTTERFLY CLIPS


The recommended items are relevant for this customer as well.

Let's now run the 100 latent factors model.

In [40]:
recommend_items(12355, training_set, user_vecs_100, item_vecs_100, customers_arr, products_arr, item_lookup,
                       num_items = 20)

,StockCode,Description
0,72807C,SET/3 VANILLA SCENTED CANDLE IN BOX
1,72807A,SET/3 ROSE CANDLE IN JEWELLED BOX
2,72807B,SET/3 OCEAN SCENT CANDLE JEWEL BOX
3,22197,SMALL POPCORN HOLDER
4,72802A,ROSE SCENT CANDLE IN JEWELLED BOX
5,72803A,ROSE SCENT CANDLE JEWELLED DRAWER
6,22969,HOMEMADE JAM SCENTED CANDLES
7,22064,PINK DOUGHNUT TRINKET POT
8,23247,BISCUIT TIN 50'S CHRISTMAS
9,23310,BUBBLEGUM RING ASSORTED


And the 100 latent factor model recommended list seems better once again.

Other methods to try:
1. recommend_all() - This method recommends items to all users at once rather than doing for just one user. I did not use it because it was easier to just pick one user and check the results and do comparisons for that user only.
2. similar_items() - It returns a list of items similar to a given item that is passed as a parameter.
3. similar_users() - It returns a list of users similar to a given user that is passed as a parameter.

## Conclusion
We have tried out one algorithm, the Alternating Least Squares implementation by the Implicit library. We saw how it was extremely fast. The library has many other amazing implementations. The readers are encouraged to experiment with those too.

## References
https://implicit.readthedocs.io/en/latest/

https://implicit.readthedocs.io/en/latest/als.html

https://github.com/benfred/implicit

https://towardsdatascience.com/recommending-github-repositories-with-google-bigquery-and-the-implicit-library-e6cce666c77

https://jessesw.com/Rec-System/